In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
from pynq import Overlay
import cv2
import os
import time

# Cargar el overlay
base = BaseOverlay("base_wrapper.bit")

# Configurar la salida HDMI
Mode = VideoMode(640, 480, 24)  # Modo del monitor: 640x480 @ 60Hz
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode, PIXEL_BGR)
hdmi_out.start()

# Configuración de la cámara (entrada)
frame_in_w, frame_in_h = 640, 480

# Establecer nivel de registro de OpenCV en silencioso
os.environ["OPENCV_LOG_LEVEL"] = "SILENT"

# Liberar cualquier recurso de cámara existente
cv2.VideoCapture(0).release()

# Inicialización de variables para el cálculo de frames por segundo
start_time = time.time()
num_frames = 0
read_errors = 0

try:
    # Inicializar la cámara
    videoIn = cv2.VideoCapture(0)
    videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
    videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)

    while True:
        # Capturar video frame por frame
        ret, frame_vga = videoIn.read()

        if ret:
            # Incrementar el contador de frames
            num_frames += 1

            # Aquí se aplica el procesamiento de imagen si es necesario
            processed_frame = frame_vga

            # Mostrar el frame procesado a través de HDMI Out
            outframe = hdmi_out.newframe()
            outframe[0:frame_in_h, 0:frame_in_w, :] = processed_frame
            hdmi_out.writeframe(outframe)
        else:
            # Incrementar el contador de errores de lectura
            read_errors += 1
            print("Error al leer de la cámara.")
            # break

except Exception as e:
    print(f"Ha ocurrido un error: {e}")

finally:
    # Calcular el tiempo transcurrido
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Calcular los frames por segundo
    fps = (num_frames - read_errors) / elapsed_time

    # Mostrar resultados
    print("Break signal...")
    print("Frames per second: " + str(fps))
    print("Number of read errors: " + str(read_errors))

    # Liberar recursos
    videoIn.release()
    hdmi_out.stop()
    hdmi_out.close()


[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
